## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 16th Mar. 23rd


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-20-2025,Condo/Co-op,6001 SW 70th St #352,South Miami,FL,33143.0,375000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/South-Miami/6001-SW-...,MARMLS,A11760858,N,Y,25.706558,-80.291032


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-20    27
2025-03-17    27
2025-03-19    22
2025-03-18    22
2025-03-21    17
2025-03-22     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
39,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715
67,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11697980,N,Y,25.752160,-80.199342
81,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11682079,N,Y,25.802013,-80.185776
45,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11730260,N,Y,25.797901,-80.127592
108,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11613016,N,Y,25.964687,-80.125368
46,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11730197,N,Y,25.772642,-80.133687
23,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,MARMLS,A11729693,N,Y,25.687302,-80.162416
17,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2100-Par...,MARMLS,A11741771,N,Y,25.797383,-80.130933
22,PAST SALE,2025-03-19,Condo/Co-op,200 Biscayne Boulevard Way Ph 5101,Miami,FL,33131.0,1700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11722556,N,Y,25.770430,-80.189163
83,PAST SALE,2025-03-21,Condo/Co-op,851 NE 1st Ave #2411,Miami,FL,33132.0,1670000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/851-NE-1st-Ave...,MARMLS,A11674698,N,Y,25.782453,-80.191836


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,...,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None
1,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,...,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Esther Sultan Osechas,"Compass Florida, LLC",None,None,Maria Lopez Pulido,Property Partners RE LLC,None,None
2,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,...,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Michael Light,Douglas Elliman,Jaimee Light,Douglas Elliman,Julie Sidorevskaya,"Julies Realty, LLC",None,None
3,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,...,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Sivan Erten,Erten Realty Group,None,None
4,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,...,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,Xavier Paulson,"HOMERICH, LLC",None,None,Angela Matusian,Miami Residence Inc.,None,None
5,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,...,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Peter Green,Fortune Christie's International Real Estate,Harry Baxter,Fortune Christie's International Real Estate,Lisa Yanowitz,Luxe Realty Group LLC,None,None
6,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,...,-80.162416,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,Isabel Dohse,"Compass Florida, LLC",None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None
7,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,...,-80.130933,https://www.redfin.com/FL/Miami-Beach/2100-Par...,Letizia Pallozzi,Miami Properties Int'l Corp,None,None,Matt McMillan,Coldwell Banker Realty,None,None
8,PAST SALE,2025-03-19,Condo/Co-op,200 Biscayne Boulevard Way Ph 5101,Miami,FL,33131.0,1700000.0,3.0,3.5,...,-80.189163,https://www.redfin.com/FL/Miami/200-Biscayne-B...,Denise Grillo,Realty One Group Plus,None,None,Ignacio Fernandez De Cabo,Affinity International Realty,None,None
9,PAST SALE,2025-03-21,Condo/Co-op,851 NE 1st Ave #2411,Miami,FL,33132.0,1670000.0,4.0,4.0,...,-80.191836,https://www.redfin.com/FL/Miami/851-NE-1st-Ave...,Madison Clivilles,"Compass Florida, LLC.",Joanna Jimenez,"Compass Florida, LLC.",Angelica Gomez,Majestic Properties LLC,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

116


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$682,803


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$536


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$79,205,150


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
135
-----------
Input Previous Week Condo Average Sales Price:
1312551
-----------
Input Previous Week Condo Average PSF:
685
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
177194410.0


In [26]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,135,1312551,685,177194410.0


In [27]:
last_week['Last Week Total Dollar Volume'].iloc[0]

177194410.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          116              682803          536                     79205150.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange
1,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,BRISTOL TOWER CONDO,2600.0,NaN,1993.0,NaN,1154.0,4388.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11697980,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Esther Sultan Osechas,"Compass Florida, LLC",None,None,Maria Lopez Pulido,Property Partners RE LLC,None,None,2,blue
2,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,Missoni Baia,2575.0,NaN,2023.0,NaN,1146.0,7666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11682079,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Michael Light,Douglas Elliman,Jaimee Light,Douglas Elliman,Julie Sidorevskaya,"Julies Realty, LLC",None,None,3,blue
3,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,2201 COLLINS AVE CONDO,1026.0,NaN,2009.0,NaN,2534.0,4357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11730260,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Sivan Erten,Erten Realty Group,None,None,4,blue
4,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,HANMPTONS SOUTH,5958.0,NaN,2004.0,NaN,431.0,6279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11613016,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,Xavier Paulson,"HOMERICH, LLC",None,None,Angela Matusian,Miami Residence Inc.,None,None,5,blue
5,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,300 COLLINS CONDO,1329.0,NaN,2018.0,NaN,1919.0,3513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11730197,N,Y,25.772642,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Peter Green,Fortune Christie's International Real Estate,Harry Baxter,Fortune Christie's International Real Estate,Lisa Yanowitz,Luxe Realty Group LLC,None,None,6,blue
6,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,LAKE VILLA TWO CONDO,2060.0,NaN,1996.0,NaN,1000.0,1754.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,MARMLS,A11729693,N,Y,25.687302,-80.162416,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,Isabel Dohse,"Compass Florida, LLC",None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,7,blue
7,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,ARTEPARK SOUTH CONDO,1442.0,NaN,2011.0,NaN,1283.0,2083.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2100-Par...,MARMLS,A11741771,N,Y,25.797383,-80.130933,https://www.redfin.com/FL/Miami-Beach/2100-Par...,Letizia Pallozzi,Miami Properties Int'l Corp,None,None,Matt McMillan,Coldwell Banker Realty,None,None,8,blue
8,PAST SALE,2025-03-19,Condo/Co-op,200 Biscayne Boulevard Way Ph 5101,Miami,FL,33131.0,1700000.0,3.0,

In [36]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_03242024


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit 406S in Key Biscayne'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,BRISTOL TOWER CONDO,2600.0,NaN,1993.0,NaN,1154.0,4388.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11697980,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Esther Sultan Osechas,"Compass Florida, LLC",None,None,Maria Lopez Pulido,Property Partners RE LLC,None,None,2,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1901...
2,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,Missoni Baia,2575.0,NaN,2023.0,NaN,1146.0,7666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11682079,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Michael Light,Douglas Elliman,Jaimee Light,Douglas Elliman,Julie Sidorevskaya,"Julies Realty, LLC",None,None,3,blue,Missoni Baia at 700 NE 26th Ter #1401 in Miami
3,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,2201 COLLINS AVE CONDO,1026.0,NaN,2009.0,NaN,2534.0,4357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11730260,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Sivan Erten,Erten Realty Group,None,None,4,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave #12...
4,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,HANMPTONS SOUTH,5958.0,NaN,2004.0,NaN,431.0,6279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11613016,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,Xavier Paulson,"HOMERICH, LLC",None,None,Angela Matusian,Miami Residence Inc.,None,None,5,blue,HANMPTONS SOUTH at 20201 E Country Club Dr Ph ...
5,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,300 COLLINS CONDO,1329.0,NaN,2018.0,NaN,1919.0,3513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11730197,N,Y,25.772642,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Peter Green,Fortune Christie's International Real Estate,Harry Baxter,Fortune Christie's International Real Estate,Lisa Yanowitz,Luxe Realty Group LLC,None,None,6,blue,300 COLLINS CONDO at 300 Collins Ave Unit 3E i...
6,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,LAKE VILLA TWO CONDO,2060.0,NaN,1996.0,NaN,1000.0,1754.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,MARMLS,A11729693,N,Y,25.687302,-80.162416,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,Isabel Dohse,"Compass Florida, LLC",None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,7,blue,LAKE VILLA TWO CONDO at 733 Crandon Blvd Ph -5...
7,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,ARTEPARK SOUTH CONDO,1442.0,NaN,2011.0,NaN,1283.0,2083.0,Sold,NaN

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,670,000 to $5,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Key Biscayne, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 116 condo sales totaling $79,205,150 million from Mar. 16th to Mar. 23rd. The previous week, brokers closed 135 condo sales totaling $177,194,410.

Last week’s units sold for an average of $682,803, lower than the $1,312,551 average price from the 

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,BRISTOL TOWER CONDO,2600.0,NaN,1993.0,NaN,1154.0,4388.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11697980,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Esther Sultan Osechas,"Compass Florida, LLC",None,None,Maria Lopez Pulido,Property Partners RE LLC,None,None,2,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1901...
2,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,Missoni Baia,2575.0,NaN,2023.0,NaN,1146.0,7666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11682079,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Michael Light,Douglas Elliman,Jaimee Light,Douglas Elliman,Julie Sidorevskaya,"Julies Realty, LLC",None,None,3,blue,Missoni Baia at 700 NE 26th Ter #1401 in Miami
3,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,2201 COLLINS AVE CONDO,1026.0,NaN,2009.0,NaN,2534.0,4357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11730260,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Sivan Erten,Erten Realty Group,None,None,4,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave #12...
4,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,HANMPTONS SOUTH,5958.0,NaN,2004.0,NaN,431.0,6279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11613016,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,Xavier Paulson,"HOMERICH, LLC",None,None,Angela Matusian,Miami Residence Inc.,None,None,5,blue,HANMPTONS SOUTH at 20201 E Country Club Dr Ph ...
5,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,300 COLLINS CONDO,1329.0,NaN,2018.0,NaN,1919.0,3513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11730197,N,Y,25.772642,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Peter Green,Fortune Christie's International Real Estate,Harry Baxter,Fortune Christie's International Real Estate,Lisa Yanowitz,Luxe Realty Group LLC,None,None,6,blue,300 COLLINS CONDO at 300 Collins Ave Unit 3E i...
6,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,LAKE VILLA TWO CONDO,2060.0,NaN,1996.0,NaN,1000.0,1754.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,MARMLS,A11729693,N,Y,25.687302,-80.162416,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,Isabel Dohse,"Compass Florida, LLC",None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,7,blue,LAKE VILLA TWO CONDO at 733 Crandon Blvd Ph -5...
7,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,ARTEPARK SOUTH CONDO,1442.0,NaN,2011.0,NaN,1283.0,2083.0,Sold,NaN

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: OCEANA KEY BISCAYNE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,670,000 to $5,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Key Biscayne, Aventura, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 116 condo sales totaling $79,205,150 million from Mar. 16th to Mar. 23rd. The previous week, brokers closed 135 condo sales totaling $177,194,410.

Last week’s units sold for an average of $682,803, lower than the $1,312,551 average price from the 

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-406S/home/79133295


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/2127-Brickell-Ave-33129/unit-1901/home/42758965


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/851-NE-1st-Ave-33132/unit-2411/home/171960605


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/200-Biscayne-Boulevard-Way-33131/unit-5101/home/43409793


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/2201-Collins-Ave-33139/unit-1214/home/43418940


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/20201-E-Country-Club-Dr-33180/unit-607/home/43317351


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-17,Condo/Co-op,360 Ocean Dr Unit 406S,Key Biscayne,FL,33149.0,5500000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2437.0,3712.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11727077,N,Y,25.694584,-80.157715,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,Gloria Ramirez,Great Properties International LLC,None,None,Jill Eber,Coldwell Banker Realty,None,None,1,orange,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
1,PAST SALE,2025-03-17,Condo/Co-op,2127 Brickell Ave #1901,Miami,FL,33129.0,3000000.0,3.0,3.0,BRISTOL TOWER CONDO,2600.0,NaN,1993.0,NaN,1154.0,4388.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11697980,N,Y,25.752160,-80.199342,https://www.redfin.com/FL/Miami/2127-Brickell-...,Esther Sultan Osechas,"Compass Florida, LLC",None,None,Maria Lopez Pulido,Property Partners RE LLC,None,None,2,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1901...
2,PAST SALE,2025-03-21,Condo/Co-op,700 NE 26th Ter #1401,Miami,FL,33137.0,2950000.0,3.0,4.5,Missoni Baia,2575.0,NaN,2023.0,NaN,1146.0,7666.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11682079,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Michael Light,Douglas Elliman,Jaimee Light,Douglas Elliman,Julie Sidorevskaya,"Julies Realty, LLC",None,None,3,blue,Missoni Baia at 700 NE 26th Ter #1401 in Miami
3,PAST SALE,2025-03-17,Condo/Co-op,2201 Collins Ave #1214,Miami Beach,FL,33139.0,2600000.0,1.0,1.5,2201 COLLINS AVE CONDO,1026.0,NaN,2009.0,NaN,2534.0,4357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11730260,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Sivan Erten,Erten Realty Group,None,None,4,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave #12...
4,PAST SALE,2025-03-20,Condo/Co-op,20201 E Country Club Dr Ph 06/07,Aventura,FL,33180.0,2570000.0,6.0,6.5,HANMPTONS SOUTH,5958.0,NaN,2004.0,NaN,431.0,6279.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20201-E-Countr...,MARMLS,A11613016,N,Y,25.964687,-80.125368,https://www.redfin.com/FL/Miami/20201-E-Countr...,Xavier Paulson,"HOMERICH, LLC",None,None,Angela Matusian,Miami Residence Inc.,None,None,5,blue,HANMPTONS SOUTH at 20201 E Country Club Dr Ph ...
5,PAST SALE,2025-03-17,Condo/Co-op,300 Collins Ave Unit 3E,Miami Beach,FL,33139.0,2550000.0,2.0,2.0,300 COLLINS CONDO,1329.0,NaN,2018.0,NaN,1919.0,3513.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11730197,N,Y,25.772642,-80.133687,https://www.redfin.com/FL/Miami-Beach/300-Coll...,Peter Green,Fortune Christie's International Real Estate,Harry Baxter,Fortune Christie's International Real Estate,Lisa Yanowitz,Luxe Realty Group LLC,None,None,6,blue,300 COLLINS CONDO at 300 Collins Ave Unit 3E i...
6,PAST SALE,2025-03-21,Condo/Co-op,733 Crandon Blvd Ph -5,Key Biscayne,FL,33149.0,2060000.0,2.0,2.5,LAKE VILLA TWO CONDO,2060.0,NaN,1996.0,NaN,1000.0,1754.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,MARMLS,A11729693,N,Y,25.687302,-80.162416,https://www.redfin.com/FL/Key-Biscayne/733-Cra...,Isabel Dohse,"Compass Florida, LLC",None,None,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,7,blue,LAKE VILLA TWO CONDO at 733 Crandon Blvd Ph -5...
7,PAST SALE,2025-03-17,Condo/Co-op,2100 Park Ave #501,Miami Beach,FL,33139.0,1850000.0,3.0,2.0,ARTEPARK SOUTH CONDO,1442.0,NaN,2011.0,NaN,1283.0,2083.0,Sold,NaN

## Time on Market Calculator

In [63]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 22) ## List (Earlier) date
date2 = datetime(2025, 3, 17) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

115


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df['$/SQUARE FEET'].astype(float)

0    2437.0
1    1154.0
2    1146.0
3    2534.0
4     431.0
5    1919.0
6    1000.0
7    1283.0
8     692.0
9     763.0
Name: $/SQUARE FEET, dtype: float64

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-17,360 Ocean Dr Unit 406S in Key Biscayne,"$5,500,000",3,4.5,"2,257",2014,"$2,437",MARMLS,A11727077,Gloria Ramirez Great Properties International ...,Jill Eber Coldwell Banker Realty
1,2025-03-17,2127 Brickell Ave #1901 in Miami,"$3,000,000",3,3.0,"2,600",1993,"$1,154",MARMLS,A11697980,"Esther Sultan Osechas Compass Florida, LLC",Maria Lopez Pulido Property Partners RE LLC
2,2025-03-21,700 NE 26th Ter #1401 in Miami,"$2,950,000",3,4.5,"2,575",2023,"$1,146",MARMLS,A11682079,Michael Light Douglas Elliman Jaimee Light Dou...,"Julie Sidorevskaya Julies Realty, LLC"
3,2025-03-17,2201 Collins Ave #1214 in Miami Beach,"$2,600,000",1,1.5,"1,026",2009,"$2,534",MARMLS,A11730260,Wendy Cohen One Sotheby's International Realty...,Sivan Erten Erten Realty Group
4,2025-03-20,20201 E Country Club Dr Ph 06/07 in Aventura,"$2,570,000",6,6.5,"5,958",2004,$431,MARMLS,A11613016,"Xavier Paulson HOMERICH, LLC",Angela Matusian Miami Residence Inc.
5,2025-03-17,300 Collins Ave Unit 3E in Miami Beach,"$2,550,000",2,2.0,"1,329",2018,"$1,919",MARMLS,A11730197,Peter Green Fortune Christie's International R...,Lisa Yanowitz Luxe Realty Group LLC
6,2025-03-21,733 Crandon Blvd Ph -5 in Key Biscayne,"$2,060,000",2,2.5,"2,060",1996,"$1,000",MARMLS,A11729693,"Isabel Dohse Compass Florida, LLC","Victor Florencio Sanchez Ocean Club Realty, In..."
7,2025-03-17,2100 Park Ave #501 in Miami Beach,"$1,850,000",3,2.0,"1,442",2011,"$1,283",MARMLS,A11741771,Letizia Pallozzi Miami Properties Int'l Corp,Matt McMillan Coldwell Banker Realty
8,2025-03-19,200 Biscayne Boulevard Way Ph 5101 in Miami,"$1,700,000",3,3.5,"2,457",2008,$692,MARMLS,A11722556,Denise Grillo Realty One Group Plus,Ignacio Fernandez De Cabo Affinity Internation...
9,2025-03-21,851 NE 1st Ave #2411 in Miami,"$1,670,000",4,4.0,"2,188",2019,$763,MARMLS,A11674698,"Madison Clivilles Compass Florida, LLC. Joanna...",Angelica Gomez Majestic Properties LLC
